# 🎓 Spinoza Secours - Notebook Colab

**Modèle :** Mistral 7B + LoRA Fine-tuned  
**Prompt :** Système hybride optimisé (~250-300 tokens)  
**API :** FastAPI + ngrok


## 📦 Cellule 1 : Installation Dépendances


In [ ]:
!pip install -q pyngrok fastapi uvicorn transformers peft accelerate bitsandbytes torch


## 📚 Cellule 2 : Imports + Configuration

**🔐 Configuration des secrets (à faire une seule fois) :**

1. Clique sur l'icône 🔑 **Secrets** dans le panneau de gauche de Colab
2. Ajoute deux secrets :
   - **`ngrok`** : Ton token ngrok (https://dashboard.ngrok.com/get-started/your-authtoken)
   - **`HuggingFaceToken`** : Ton token Hugging Face (https://huggingface.co/settings/tokens)

Les tokens seront récupérés automatiquement via `userdata.get()`.


In [ ]:
import os
import re
import random
import threading
import torch
from typing import Dict, List, Optional
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import userdata

# Configuration modèle
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_MODEL = "FJDaz/mistral-7b-philosophes-lora"

# Récupération des tokens depuis Colab Secrets
# ⚠️ Configure tes secrets dans Colab : 🔑 Secrets → Ajouter 'ngrok' et 'HuggingFaceToken'
try:
    NGROK_TOKEN = userdata.get('ngrok')
    print("✅ Token ngrok récupéré depuis Colab Secrets")
except:
    print("❌ Token ngrok non trouvé ! Configure le secret 'ngrok' dans Colab Secrets")
    NGROK_TOKEN = None

try:
    HF_TOKEN = userdata.get('HuggingFaceToken')
    print("✅ Token Hugging Face récupéré depuis Colab Secrets")
except:
    print("❌ Token Hugging Face non trouvé ! Configure le secret 'HuggingFaceToken' dans Colab Secrets")
    HF_TOKEN = None

# Configuration ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("⚠️ ngrok ne sera pas configuré sans token")

# Gestion des ports : Port aléatoire pour éviter les conflits
PORT = random.randint(8001, 9000)
print(f"🔌 Port sélectionné : {PORT}")

# Tuer processus sur le port sélectionné si existant
import subprocess
try:
    result = subprocess.run(f"lsof -ti:{PORT}", shell=True, capture_output=True, text=True)
    if result.stdout.strip():
        subprocess.run(f"kill -9 {result.stdout.strip()}", shell=True)
        print(f"🧹 Port {PORT} libéré")
except:
    pass

print("✅ Imports et configuration OK")


## 🎯 Cellule 3 : Prompt Système Hybride


In [ ]:
# Prompt système hybride optimisé (~250 tokens)
SYSTEM_PROMPT_SPINOZA = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

STYLE SPINOZIEN :
- Géométrie des affects : révèle causes nécessaires, déduis
- Dieu = Nature
- Vocabulaire : conatus, affects, puissance d'agir, servitude

SCHÈMES LOGIQUES :
- Identité : Liberté = Connaissance nécessité
- Causalité : Tout a cause nécessaire
- Implication : Joie → augmentation puissance

MÉTHODE :
1. Révèle nécessité causale
2. Distingue servitude (ignorance) vs liberté (connaissance)
3. Exemples concrets modernes

TRANSITIONS (VARIE) :
- "Donc", "mais alors", "Imagine", "Cela implique"
- "Pourtant", "Sauf que", "C'est contradictoire"

RÈGLES :
- Tutoie (tu/ton/ta)
- Concis (2-3 phrases MAX)
- Questionne au lieu d'affirmer
- Ne parle JAMAIS de toi à la 3ème personne. Tu ES Spinoza.
- Vocabulaire MODERNE : utilise le langage d'aujourd'hui, évite les archaïsmes
- Si tu utilises "conatus", "affects", explique-les avec des mots d'aujourd'hui"""

INSTRUCTIONS_CONTEXTUELLES = {
    "confusion": "L'élève est confus → Donne UNE analogie concrète simple en utilisant tes schèmes logiques.",
    "resistance": "L'élève résiste → Révèle contradiction avec 'mais alors' et tes schèmes logiques.",
    "accord": "L'élève est d'accord → Valide puis AVANCE logiquement avec 'Donc' et tes schèmes logiques.",
    "neutre": "Élève neutre → Pose question pour faire réfléchir en utilisant tes schèmes logiques."
}

INSTRUCTION_RAG = """
UTILISATION CONNAISSANCES :
- Tu connais l'Éthique de Spinoza
- Cite implicitement ("comme je l'ai montré...", "dans mon œuvre...")
- Reformule dans TON style (première personne, lycéen)
- Ne récite pas : extrais idées et reformule naturellement
"""

def construire_prompt_complet(contexte: str, use_rag_instruction: bool = True) -> str:
    """
    Construit le prompt complet optimisé
    
    Args:
        contexte: "accord", "confusion", "resistance", "neutre"
        use_rag_instruction: Si True, ajoute instructions RAG
    
    Returns:
        Prompt système complet (~250-300 tokens)
    """
    prompt = SYSTEM_PROMPT_SPINOZA
    
    # Ajouter instruction contextuelle
    if contexte in INSTRUCTIONS_CONTEXTUELLES:
        prompt += f"\n\n{INSTRUCTIONS_CONTEXTUELLES[contexte]}"
    
    # Ajouter instruction RAG (optionnel)
    if use_rag_instruction:
        prompt += f"\n\n{INSTRUCTION_RAG}"
    
    return prompt

print("✅ Prompt système hybride chargé")


## 🔍 Cellule 4 : Détection Contexte + Post-Processing


In [ ]:
def detecter_contexte(user_input: str) -> str:
    """Détecte le contexte de la réponse utilisateur"""
    text_lower = user_input.lower()
    
    # Accord
    if any(word in text_lower for word in ['oui', "d'accord", 'exact', 'ok', 'voilà', 'tout à fait']):
        return "accord"
    
    # Confusion
    if any(phrase in text_lower for phrase in ['comprends pas', 'vois pas', "c'est quoi", 'je sais pas', 'pourquoi', 'rapport']):
        return "confusion"
    
    # Résistance
    if any(word in text_lower for word in ['mais', 'non', "pas d'accord", 'faux', "n'importe quoi", 'je peux']):
        return "resistance"
    
    return "neutre"

def nettoyer_reponse(text: str) -> str:
    """Nettoie la réponse générée"""
    text = re.sub(r'\([^)]*[Aa]ttends[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Pp]oursuis[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Dd]onne[^)]*\)', '', text)
    text = re.sub(r'[😀-🙏🌀-🗿🚀-🛿]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+([.!?])', r'\1', text)
    return text

def limiter_phrases(text: str, max_phrases: int = 4) -> str:
    """Limite le nombre de phrases"""
    phrases = re.split(r'[.!?]+\s+', text)
    phrases = [p.strip() for p in phrases if p.strip()]
    if len(phrases) <= max_phrases:
        return text
    return '. '.join(phrases[:max_phrases]) + '.'

print("✅ Détection contexte + Post-processing OK")


## 🤖 Cellule 5 : Chargement Modèle

**⚠️ Code reproduit exactement depuis app.py - Ne pas modifier**


In [ ]:
@torch.no_grad()
def load_model():
    has_gpu = torch.cuda.is_available()
    print(f"🖥️ GPU disponible: {has_gpu}")

    if has_gpu:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        device_map = "auto"
        torch_dtype = torch.bfloat16
    else:
        quantization_config = None
        device_map = "cpu"
        torch_dtype = torch.float32

    print(f"🔄 Chargement Mistral 7B ({'4-bit GPU' if has_gpu else 'FP32 CPU'})...")

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=quantization_config,
        device_map=device_map,
        torch_dtype=torch_dtype,
        token=HF_TOKEN,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    print("🔄 Chargement tokenizer...")

    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL,
        token=HF_TOKEN,
        trust_remote_code=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("🔄 Application LoRA Spinoza_Secours...")

    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_MODEL,
        token=HF_TOKEN
    )

    print("✅ Modèle Mistral 7B + LoRA chargé!")
    return model, tokenizer

# Charger le modèle
model, tokenizer = load_model()


## 💬 Cellule 6 : Fonction spinoza_repond()

**Utilise le prompt système hybride optimisé**


In [ ]:
# Historique conversation (format: [user, assistant])
conversation_history = []

def spinoza_repond(message: str) -> str:
    """
    Génère une réponse de Spinoza avec prompt hybride adaptatif
    
    Args:
        message: Message de l'utilisateur
    
    Returns:
        Réponse de Spinoza nettoyée
    """
    global conversation_history
    
    # Détecter contexte
    contexte = detecter_contexte(message)
    
    # Construire prompt adaptatif (RAG désactivé par défaut - trop embrouillant)
    system_prompt = construire_prompt_complet(contexte, use_rag_instruction=False)
    
    # Formatage Mistral style
    prompt_parts = [f"<s>[INST] {system_prompt}\n\n"]
    
    # Ajouter historique (4 derniers échanges max)
    for entry in conversation_history[-4:]:
        prompt_parts.append(f"{entry[0]} [/INST] {entry[1]}</s>[INST] ")
    
    prompt_parts.append(f"{message} [/INST]")
    text = "".join(prompt_parts)
    
    # Tokenization
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]
    
    # Génération
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32
    
    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Décodage
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    # Post-processing
    response = nettoyer_reponse(response)
    response = limiter_phrases(response, max_phrases=3)
    
    # Mettre à jour historique
    conversation_history.append([message, response])
    
    return response

print("✅ Fonction spinoza_repond() créée")


## 🌐 Cellule 7 : API FastAPI + ngrok


In [ ]:
# Questions d'amorce
QUESTIONS_BAC = [
    "La liberté est-elle une illusion ?",
    "Suis-je esclave de mes désirs ?",
    "Puis-je maîtriser mes émotions ?",
    "La joie procure-t-elle un pouvoir ?",
    "Peut-on désirer sans souffrir ?",
]

# Modèles Pydantic
class ChatRequest(BaseModel):
    message: str
    history: Optional[List[List[str]]] = None

class ChatResponse(BaseModel):
    reply: str
    history: List[List[str]]

# API FastAPI
app = FastAPI()

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # ⚠️ À restreindre en production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def root():
    """Endpoint racine - Informations sur l'API"""
    return {
        "name": "Spinoza Secours API",
        "model": "Mistral 7B + LoRA",
        "status": "running",
        "endpoints": {
            "health": "GET /health",
            "init": "GET /init",
            "chat": "POST /chat"
        }
    }

@app.get("/health")
def health():
    return {"status": "ok", "model": "Mistral 7B + LoRA"}

@app.get("/init")
def init():
    global conversation_history
    conversation_history = []
    question = random.choice(QUESTIONS_BAC)
    greeting = f"Bonjour ! Je suis Spinoza. Discutons :\n\n**{question}**\n\nQu'en penses-tu ?"
    return {
        "greeting": greeting,
        "history": [[None, greeting]]
    }

@app.post("/chat")
def chat(req: ChatRequest):
    global conversation_history
    
    # Mettre à jour historique si fourni
    if req.history:
        conversation_history = req.history
    
    # Générer réponse
    reply = spinoza_repond(req.message)
    
    return {
        "reply": reply,
        "history": conversation_history
    }

print("✅ API FastAPI créée")


## 🚀 Cellule 8 : Lancement Serveur + ngrok


In [ ]:
# Lancer FastAPI en background avec port dynamique
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Attendre que le serveur démarre
import time
time.sleep(3)

# Tunnel ngrok sur le port dynamique
tunnel = ngrok.connect(PORT)
# Extraire l'URL publique depuis l'objet NgrokTunnel
# L'objet NgrokTunnel a une méthode public_url ou on peut extraire depuis str()
if hasattr(tunnel, 'public_url'):
    public_url = tunnel.public_url
elif hasattr(tunnel, 'data') and 'public_url' in tunnel.data:
    public_url = tunnel.data['public_url']
else:
    # Fallback : extraire depuis la représentation string
    import re
    url_match = re.search(r'https://[^"]+\.ngrok[^"]+', str(tunnel))
    public_url = url_match.group(0) if url_match else str(tunnel)

print("\n" + "=" * 80)
print(f"🚀 API PUBLIQUE : {public_url}")
print(f"📡 Health : {public_url}/health")
print(f"💬 Init : {public_url}/init")
print(f"💬 Chat : POST {public_url}/chat")
print("=" * 80)
print(f"\n✅ Serveur lancé sur le port {PORT} !")
print(f"📋 Copie cette URL dans index_spinoza.html : {public_url}")
